In [ ]:
# Install dependencies and libraries
!pip install -r requirements.txt

In [5]:
# Import libraries
import pandas as pd
import numpy as np
import xlwings as xw
import tempfile
import datetime
# For Barcode generation
from barcode import EAN13
# import ImageWriter to generate an image file
from barcode.writer import ImageWriter
# For QR Code generation
import segno

## Get the Workspace One Data Dump


In [6]:
# TODO: Automatically get the filename from the dump
fileName = "Grid_Report_export_WS_143b193c-8e37-43ee-824d-50e9e16bfcd7.csv"
df_dump = pd.read_csv("./dump/" + fileName)

# Summary information
df_dump.info()
df_dump.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Last Seen           104 non-null    object 
 1   Friendly Name       104 non-null    object 
 2   Ownership           104 non-null    object 
 3   Username            93 non-null     object 
 4   First Name          93 non-null     object 
 5   Last Name           93 non-null     object 
 6   Email               93 non-null     object 
 7   Platform            104 non-null    object 
 8   OS                  104 non-null    object 
 9   Model               104 non-null    object 
 10  Phone               0 non-null      float64
 11  Tags                83 non-null     object 
 12  Enrollment          104 non-null    object 
 13  Compliance Status   104 non-null    object 
 14  Display Name        93 non-null     object 
 15  Management          104 non-null    object 
 16  Asset Nu

,Last Seen,Friendly Name,Ownership,Username,First Name,Last Name,Email,Platform,OS,Model,...,Wi-Fi IP Address,Public IP Address,Wi-Fi MAC Address,Notes,WNS Status,DM Last Seen,Build Version,Wi-Fi SSID,Android Management,CPU Architecture
0,9/24/2021 5:32:02 PM,CNHKG03-WIW016,C,chunpong.chan,Chunpong,Chan,chunpong.chan@hk.rhenus.com,WinRT,10.0.19043,OptiPlex 3070,...,NaN,203.198.186.172,50:EB:71:BF:7B:56,False,True,9/24/2021 5:32:02 PM,1237,NaN,NaN,X64
1,9/24/2021 5:29:52 PM,CNHKG01-WIL037,C,ivy.ip,Ivy,Ip,ivy.ip@hk.rhenus.com,WinRT,10.0.19041,Latitude 7480,...,NaN,118.140.216.98,74:E5:F9:F0:BD:BF,False,True,9/24/2021 5:29:59 PM,1165,NaN,NaN,X64
2,9/24/2021 5:29:01 PM,CNHKG04-WIL004,C,Marco.So,Marco,So,Marco.So@hk.rhenus.com,WinRT,10.0.19043,Latitude 5420,...,NaN,203.198.186.172,10:3D:1C:39:82:C1,False,True,9/24/2021 2:28:02 PM,1237,NaN,NaN,X64
3,9/24/2021 5:24:16 PM,CNHKG01-WIW045,C,CNHKG01.Shared,CNHKG01,Shared,CNHKG01.Shared@hk.rhenus.com,WinRT,10.0.19043,OptiPlex 3040,...,NaN,118.140.216.98,NaN,False,True,9/24/2021 11:34:04 AM,1237,NaN,NaN,X64
4,9/24/2021 5:24:01 PM,CNHKG04-WIW007,C,CNHKG04.Shared,CNHKG04,Shared,CNHKG04.Shared@hk.rhenus.com,WinRT,10.0.19043,OptiPlex 3070,...,NaN,203.198.186.172,AC:67:5D:14:E3:7B,False,True,9/24/2021 5:24:01 PM,1237,NaN,NaN,X64


# Get Invoices

In [7]:
# The row to start considering data from
header = 3
# Read invoices.xlsx
df_invoice = pd.read_excel("./src/Invoice.xlsx", header=header)
# Show summary information
df_invoice.info()
df_invoice.head(100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Invoice Number            23 non-null     object        
 1   Serial Number             251 non-null    object        
 2   Purchase Date             23 non-null     datetime64[ns]
 3   Warranty Expiration Date  23 non-null     datetime64[ns]
 4   Invoice File              23 non-null     object        
dtypes: datetime64[ns](2), object(3)
memory usage: 9.9+ KB


,Invoice Number,Serial Number,Purchase Date,Warranty Expiration Date,Invoice File
0,1400666873,6TF31L2,2017-08-20,2022-08-20,https://rhenusasia.sharepoint.com/:b:/s/ITMast...
1,1400666873,6TDW0L2,2017-08-20,2022-08-20,https://rhenusasia.sharepoint.com/:b:/s/ITMast...
2,1400614031,F17XPC2,2016-10-05,2021-10-05,https://rhenusasia.sharepoint.com/:b:/s/ITMast...
3,2017-4159,PF0Q6AFJ,2017-04-21,2020-04-21,https://rhenusasia.sharepoint.com/:b:/s/ITMast...
4,1400666873,6TB41L2,2017-08-20,2022-08-20,https://rhenusasia.sharepoint.com/:b:/s/ITMast...
...,...,...,...,...,...
95,NaN,1W6NMK2,NaT,NaT,NaN
96,NaN,B66NYF2,NaT,NaT,NaN
97,NaN,CWC6FH2,NaT,NaT,NaN
98,NaN,9Z2XZG2,NaT,NaT,NaN


# TODO: Clean Dump to reflect MasterData template

In [ ]:
# Specify columns to be used for local records and master data, in the right order.

# devices_columns = {"Status", "Email", "City Code", "Computer Name", "Manufacturer", "Category", "Serial Number", "Friendly Name", "Model"}
# df_device = df_dump[devices_columns]

# Merge invoice information with Workspace One dump

In [37]:
devices_columns = {"Serial Number", "Friendly Name", "Email", "Model", "Tags"}
df_device = df_dump[devices_columns]
# Join invoice information and device information
df_device_invoice = pd.merge(df_device, df_invoice, on="Serial Number", how="left")
# Drop duplicated entries
## TODO: Make sure this is representative of reality.
df_device_invoice = df_device_invoice.drop_duplicates()
# Add blank columns for QR Codes and Barcodes
df_device_invoice["Bar Code"] = ""
df_device_invoice["QR Code"] = ""
# Re-arrange columns
custom_column_order = ["QR Code", "Serial Number", "Friendly Name", "Model", "Tags", "Invoice Number", "Purchase Date", "Warranty Expiration Date", "Invoice File", "Bar Code"]
df_device_invoice = df_device_invoice[custom_column_order]
## TODO: Generalise for MasterData
# Summary information
# df_device_invoice.info()
# df_device_invoice.head(100)
df_device_invoice["Serial Number"].value_counts()


F17XPC2    2
5522HZ2    2
G39JSF2    2
2MPG573    2
CL58033    2
          ..
D2WSLQ2    1
HL60J13    1
1MRLQ13    1
HR761N2    1
DKXN0R2    1
Name: Serial Number, Length: 99, dtype: int64

# Export Device-Invoice data to Excel Workbook.

In [44]:
# Generate timestamp
current_day = str(datetime.datetime.now().year) + "_" + str(datetime.datetime.now().month) + "_" + str(datetime.datetime.now().day)
# Varible for book name
book_url = "./output/CNHKG0X_IT_ASSETS_" + current_day + ".xlsx"

df_device_invoice.to_excel(book_url, index=False)

# Generate QR Codes and Bar Codes for all devices.

In [46]:
# Update this with the name of your workbook
book = xw.Book(book_url)
sheet = book.sheets[0]
# Update this with the starting cell of your URLs
start_cell = sheet['B2']
# Get Serial numbers of devices
urls = start_cell.options(expand='down', ndim=1).value


# # # Loop through each URL and generate the QR code
for id, url in enumerate(urls):
    # print(id, url)
    # Generate the QR code
    qr = segno.make(url)
    # create an object of EAN13 class and pass the number with the ImageWriter() as the writer    

    with tempfile.TemporaryDirectory() as td:
        # Save the QR code as a temporary svg file. If you are on macOS, use pdf
        # instead and if you don't have Microsoft 365, you may have to use png
        td = str(td)
        filepath = f'{td}/qr.svg'
        qr.save(filepath, scale=5, border=0, finder_dark='#000000')
        # Insert the QR code to the right of the URL
        destination_cell = start_cell.offset(row_offset=id, column_offset=-1)
        # Add temporary image to spreadsheet
        sheet.pictures.add(filepath, left=destination_cell.left, top=destination_cell.top, width=50, height=50)

# sheet.autofit("c")
# sheet.autofit("r")
# # Resize first column to make space for QR Code



com_error: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2146827284), None)